In [1]:
# mart_cifar10.py
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.models import resnet18
import numpy as np
import random
from tqdm import tqdm

# --------------------------
# 1. Reproducibility
# --------------------------
seed = 42
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# --------------------------
# 2. Hyperparameters
# --------------------------
batch_size = 128
learning_rate = 0.01
num_epochs = 80
epsilon = 8 / 255
alpha = 2 / 255
num_steps = 10
beta = 6.0

# --------------------------
# 3. CIFAR-10 Data
# --------------------------
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
])

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

# --------------------------
# 4. Model (Pretrained)
# --------------------------
model = resnet18(weights="IMAGENET1K_V1")
model.fc = nn.Linear(model.fc.in_features, 10)
model = model.to(device)
print("Loaded pretrained ResNet-18 with ImageNet weights")

# --------------------------
# 5. MART Loss Function
# --------------------------
def mart_loss(model, x_natural, y):
    model.train()
    batch_size = len(x_natural)
    x_adv = x_natural.detach() + 0.001 * torch.randn_like(x_natural)

    # PGD inner loop
    for _ in range(num_steps):
        x_adv.requires_grad_()
        logits_adv = model(x_adv)
        loss_adv = F.cross_entropy(logits_adv, y)
        grad = torch.autograd.grad(loss_adv, [x_adv])[0]
        x_adv = x_adv + alpha * torch.sign(grad)
        x_adv = torch.min(torch.max(x_adv, x_natural - epsilon), x_natural + epsilon)
        x_adv = torch.clamp(x_adv, 0.0, 1.0).detach()

    # MART specific loss
    logits_nat = model(x_natural)
    logits_adv = model(x_adv)
    adv_probs = F.softmax(logits_adv, dim=1)
    nat_probs = F.softmax(logits_nat, dim=1)

    # True class probabilities
    true_probs = nat_probs.gather(1, y.unsqueeze(1)).squeeze()

    # Cross-entropy on adversarial examples
    loss_adv = F.cross_entropy(logits_adv, y)

    # KL term weighted by (1 - p_y)
    loss_robust = F.kl_div(F.log_softmax(logits_adv, dim=1),
                           nat_probs,
                           reduction='none').sum(dim=1)
    loss_robust = torch.mean(loss_robust * (1.0 - true_probs))

    return loss_adv + beta * loss_robust

# --------------------------
# 6. Optimizer & Scheduler
# --------------------------
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9, weight_decay=5e-4)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)

# --------------------------
# 7. Evaluation Function
# --------------------------
def evaluate(model, loader, adversarial=False):
    model.eval()
    correct = 0
    total = 0
    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)
        if adversarial:
            x_adv = images.detach() + 0.001 * torch.randn_like(images)
            for _ in range(num_steps):
                x_adv.requires_grad_()
                logits = model(x_adv)
                loss = F.cross_entropy(logits, labels)
                grad = torch.autograd.grad(loss, [x_adv])[0]
                x_adv = x_adv + alpha * torch.sign(grad)
                x_adv = torch.min(torch.max(x_adv, images - epsilon), images + epsilon)
                x_adv = torch.clamp(x_adv, 0.0, 1.0).detach()
            outputs = model(x_adv)
        else:
            outputs = model(images)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
    return 100.0 * correct / total

# --------------------------
# 8. Training Loop
# --------------------------
print("Starting MART training...")
for epoch in range(1, num_epochs + 1):
    model.train()
    running_loss = 0.0
    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch}/{num_epochs}"):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        loss = mart_loss(model, images, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * images.size(0)

    scheduler.step()
    avg_loss = running_loss / len(train_loader.dataset)
    clean_acc = evaluate(model, test_loader, adversarial=False)
    adv_acc = evaluate(model, test_loader, adversarial=True)
    print(f"Epoch [{epoch}/{num_epochs}] | Loss: {avg_loss:.4f} | Clean Acc: {clean_acc:.2f}% | Adv Acc: {adv_acc:.2f}%")

# --------------------------
# 9. Save Model
# --------------------------
torch.save(model.state_dict(), "mart_resnet18_cifar10.pth")
print("Training completed and model saved!")


Using device: cuda


100%|██████████| 170M/170M [00:02<00:00, 57.6MB/s] 
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 120MB/s] 


Loaded pretrained ResNet-18 with ImageNet weights
Starting MART training...


Epoch 1/80: 100%|██████████| 391/391 [01:53<00:00,  3.45it/s]


Epoch [1/80] | Loss: 2.4393 | Clean Acc: 10.76% | Adv Acc: 9.83%


...

Epoch [78/80] | Loss: 1.7093 | Clean Acc: 57.85% | Adv Acc: 40.14%


Epoch 79/80: 100%|██████████| 391/391 [01:51<00:00,  3.51it/s]


Epoch [79/80] | Loss: 1.7122 | Clean Acc: 58.29% | Adv Acc: 39.93%


Epoch 80/80: 100%|██████████| 391/391 [01:51<00:00,  3.51it/s]


Epoch [80/80] | Loss: 1.7068 | Clean Acc: 58.40% | Adv Acc: 39.81%
Training completed and model saved!


In [2]:
torch.save(model.state_dict(), "mart_resnet18_cifar10.pth")
print("Training completed and model saved!")

Training completed and model saved!
